In [1]:
from lively import CollisionAvoidanceObjective, PositionLivelinessObjective, Solver, Translation, Rotation, Transform, SmoothnessMacroObjective, BoxShape, CollisionSettingInfo, ScalarRange, OrientationLivelinessObjective
from lxml import etree

# Constructed box Shape and configured Collision Setting 
box = BoxShape(name="Table",frame="world",physical=True,x=2,y=1,z=1.2, translation = Translation(x=1.0, y =2.0, z=3.0), rotation = Rotation(x=0.0,y=0.0,z=0.0,w=1.0))
collision = CollisionSettingInfo(d_max = 0.1, r = 0.0, a_max = 2.0, time_budget = 100, timed = False)

# Read the xml file into a string
xml_file = '../../tests/panda.xml'
tree = etree.parse(xml_file)
xml_string = etree.tostring(tree).decode()

#print(xml_string)
# Instantiate a new solver
solver = Solver(
  urdf=xml_string, # Full urdf as a string
  objectives={
      # An example objectives
      "smoothness":SmoothnessMacroObjective(name="MySmoothnessObjective",weight=5,joints=True,origin=False,links=True),
      "collision": CollisionAvoidanceObjective(name="MyCollisionAvoidanceObjective", weight=5),
      # Livelyliness objectives
      "positionLiveliness": PositionLivelinessObjective(name="MyPosLivelinessObjective",link="l_shoulder1",frequency=7,weight=25),
      "orientationLiveliness": OrientationLivelinessObjective(name="MyOrtLivelinessObjective", link="l_shoulder1",frequency=7,weight=25)
      
  },
  root_bounds=[
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0), # Translational, (x, y, z)
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0)  # Rotational, (r, p, y)
  ],
  shapes=[
      box
  ],
  collision_settings = collision

)

# Run solve to get a solved state
state = solver.solve(goals= {"myGoal": Translation(x=1.0, y =2.0, z=3.0)},weights = {},time = 0.0)
# Log the initial state
print(state.origin.as_dicts())
print(state.joints)
print(state.frames["panda_link0"])
print(state.proximity)

({'z': 0.0, 'x': 0.0, 'y': 0.0}, {'y': 0.0, 'x': 0.0, 'z': 0.0, 'w': 1.0})
{'panda_finger_joint2': 0.029211291676823688, 'panda_joint5': -3.90060211223606e-08, 'panda_joint6': 1.8675023747662598, 'panda_joint7': -0.0001456668049438404, 'panda_joint1': -1.752150322748814e-07, 'panda_joint4': -1.5708393176936717, 'panda_joint2': -4.5450760628263015e-06, 'panda_finger_joint1': 0.029211291676823688, 'panda_joint3': -1.7522326112491775e-07}
TransformInfo: world:{translation: {x:0.0, y:0.0, z:0.0}, rotation: {w:1.0, x:0.0, y:0.0, z:0.0}}, local: {translation: {x:0.0, y:0.0, z:0.0}, rotation: {w:1.0, x:0.0, y:0.0, z:0.0}}
[ProximityInfo: {shape1: "panda_link1", shape2: "panda_link2", distance: -0.04999999999998843, points: Some(([-2.675236318150422e-7, 1.2481875700535767e-13, 0.3929999999993975], [-5.063169258167043e-8, 9.069134332406747e-14, 0.3429999999998795])), physical: true, loss: 1.0499999999999885, average_distance: Some(1.0)}, ProximityInfo: {shape1: "panda_link3", shape2: "panda_lin